In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [ ]:
import datetime as dt
from flask import Flask, jsonify


In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
Measurement = Base.classes.measurement
Station = Base.classes.station

session = Session(engine)

In [ ]:
app = Flask(__name__)

In [ ]:
#create homepage route
@app.route("/")
def welcome():
    """List of all returnable API routes."""
    return(
        f"/api/v1.0/precipitation<br/>"
        f"- Query temperatures for last year. <br/>"

        f"/api/v1.0/stations<br/>"
        f"- List of station. <br/>"

        f"/api/v1.0/tobs<br/>"
        f"- Temperatures for the last year. <br/>"

        f"/api/v1.0/yyyy-mm-dd/<br/>"
        f"- Average/Max/Min temperatures for a date <br/>"

        f"/api/v1.0/yyyy-mm-dd/yyyy-mm-dd/<br/>"
        f"- Average/Max/Min temperatures for a period <br/>"
    )

In [ ]:
#create query to find temperatures and dates for last year
@app.route("/api/v1.0/precipitation")
def prcp():
    """Query temperatures for last year."""
    results = session.query(Measurement.date, Measurement.tobs).\
        filter(Measurement.date <= "2016-01-01", Measurement.date >= "2016-01-01").\
        all()

    
    precipitation_list = [results]

    return jsonify(precipitation_list)


In [ ]:
#create query to find stations
@app.route("/api/v1.0/stations")
def stations():
    """List of Stations"""
    results = session.query(Station.name, Station.station, Station.elevation).all()

    station_list = []
    for result in results:
        row = {}
        row['name'] = result[0]
        row['station'] = result[1]
        row['elevation'] = result[2]
        station_list.append(row)
    return jsonify(station_list)

In [ ]:
#create query to find temperatures for last year
@app.route("/api/v1.0/tobs")
def station_temps():
    """Temperatures by station for the last year."""
    results = session.query(Station.name, Measurement.date, Measurement.tobs).\
        filter(Measurement.date >= "2016-01-01", Measurement.date <= "2017-01-01").\
        all()

    
    tobs_list = []
    for result in results:
        row = {}
        row["Date"] = result[1]
        row["Station"] = result[0]
        row["Temperature"] = int(result[2])
        tobs_list.append(row)

    return jsonify(tobs_list)

In [ ]:
#create query to avg/max/min temps for a date
@app.route('/api/v1.0/<date>/')
def avg_date(date):
    """Average/Max/Min temperatures for a date """
    results = session.query(Measurement.date, func.avg(Measurement.tobs), func.max(Measurement.tobs), func.min(Measurement.tobs)).\
        filter(Measurement.date == date).all()

    data_list = []
    for result in results:
        row = {}
        row['Date'] = result[0]
        row['Average Temperature'] = float(result[1])
        row['Highest Temperature'] = float(result[2])
        row['Lowest Temperature'] = float(result[3])
        data_list.append(row)

    return jsonify(data_list)

In [ ]:
@app.route('/api/v1.0/<start_date>/<end_date>/')
def avg_period(start_date, end_date):
    """Average/Max/Min temperatures for a period """
    results = session.query(func.avg(Measurement.tobs), func.max(Measurement.tobs), func.min(Measurement.tobs)).\
        filter(Measurement.date >= start_date, Measurement.date <= end_date).all()

    data_list = []
    for result in results:
        row = {}
        row["Start Date"] = start_date
        row["End Date"] = end_date
        row["Average Temperature"] = float(result[0])
        row["Highest Temperature"] = float(result[1])
        row["Lowest Temperature"] = float(result[2])
        data_list.append(row)
    return jsonify(data_list)

In [ ]:
if __name__ == '__main__':
    app.run()